## Setup

In [1]:
%load_ext autoreload
%autoreload 2

### Dependencies

In [2]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [3]:
import datetime as dt
import os

import ipywidgets as widgets
from IPython.display import display,clear_output

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None   # suppresses SettingWithCopyWarning

In [5]:
pwd

'C:\\Users\\timot\\Desktop\\Mila\\Courses\\IFT6758\\Project\\Project Service\\Docker\\notebooks'

In [6]:
from ift6758.client import GameTracker, NHLAPIClient, PredictorAPIClient
from ift6758.client.nhl_api_client import UnknownGameException
from ift6758.client.predictor_api_client import UnknownModelException
from ift6758.client.utils import MODEL_OPTIONS

### Setup

In [7]:
if not os.path.exists("./tmp_data/"):
    os.makedirs("./tmp_data/")

In [18]:
nhl_api = NHLAPIClient(raw_game_directory="./tmp_data/")
predictor = PredictorAPIClient(ip="serving")
#predictor = PredictorAPIClient(ip="127.0.0.1")
game_tracker = GameTracker(nhl_api_client=nhl_api, predictor_client=predictor)

### Dashboard Actions

In [19]:
def on_button_update_clicked(btn):

    with output:
        try:
            game_tracker.update(game_id_tbox.value, model_id_tbox.value)
        except UnknownGameException as uge:
            print(f"Game ID {game_id_tbox.value} not recognized, so dashboard unchanged")
        except UnknownModelException as ume:
            print(f"Dashboard unchanged because model_id " + str(ume))

        events = game_tracker.get_events_and_predictions()

        most_recent_event = events.head(1).squeeze()
        home_team = most_recent_event['home_team']
        away_team = most_recent_event['away_team']

        home_xG = round(events.loc[events['shooter_team_name']==home_team, 'predictions'].sum(), 2)
        away_xG = round(events.loc[events['shooter_team_name']==away_team, 'predictions'].sum(), 2)

        clear_output()
        print(f"Model Loaded: {model_id_tbox.value}")
        print(f"Period {most_recent_event['period']}, {most_recent_event['time_remaining']} Remaining")
        print(f"Home: {home_team} Away: {away_team}")
        print(f"Home G: {most_recent_event['goals_home']} Away G: {most_recent_event['goals_away']}")
        print(f"Home xG: {home_xG} Away xG: {away_xG}")
        print(f"Recent events:")
        display(events[['type','description', 'predictions']].head(15))


In [20]:
game_id_tbox = widgets.Text(description="game_id", width=300, value = "2021020008")
#model_id_tbox = widgets.Text(description="model_id", width=300, value = "xgboost-feats-non-corr")
model_id_tbox = widgets.Dropdown(
    options=MODEL_OPTIONS,
    value="xgboost-feats-non-corr",
    description='Number:',
)



button_update = widgets.Button(description="Update Dashboard")

button_update.on_click(on_button_update_clicked)
output = widgets.Output()

display(widgets.HBox((game_id_tbox, model_id_tbox, button_update)))
display(output)

Output()

In [16]:
# print(err_msg)
# 

# return err_msg, components


'xgboost-feats-non-corr'

In [97]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(game_tracker.events[['type', 'shooter_team_name','predictions']])

In [98]:
display(button_update)
# display(widgets.HBox((game_id_tbox, model_id_tbox, button_update)))
display(output)
# 2021020001, xgboost-feats-non-corr

NameError: name 'button_update' is not defined